In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks/apex-codes/entity_sum

/content/drive/My Drive/Colab Notebooks/apex-codes/entity_sum


## Clone stanza

In [ ]:
!git clone https://github.com/stanfordnlp/stanza.git

fatal: destination path 'stanza' already exists and is not an empty directory.


In [ ]:
!pip3 install stanza

     |████████████████████████████████| 337kB 5.0MB/s 


## https://stanfordnlp.github.io/stanza/biomed_model_usage.html

In [ ]:
import stanza
import pandas as pd
import numpy as np
from collections import defaultdict
import os
import pickle as pk

## Read the file with the medline abstracts

In [ ]:
def _get_abstracts(fpath):
  df = pd.read_excel(fpath) 
  return df

In [ ]:
def _extract_named_entities(icd_chapter, i2b2_model, bc5cdr_model):
  df = _get_abstracts(icd_chapter)
  df = df[['PMID', 'Title', 'Abstract']]

  df['Named_Entities_mimic_w_i2b2'] = ""
  df['Named_Entities_genia_w_BC5CDR'] = ""

  for idx, row in df.iterrows():
    dict_i2b2_entities = defaultdict(list)
    dict_bc5cdr_entities = defaultdict(list)

    try:
      doc_i2b2 = i2b2_model(df['Abstract'][idx])
      for ent in doc_i2b2.entities:
        dict_i2b2_entities[ent.type].append(ent.text)
      df['Named_Entities_mimic_w_i2b2'][idx] = dict_i2b2_entities
    except (AssertionError, RuntimeError):
      continue

    try:
      doc_bc5cdr = bc5cdr_model(df['Abstract'][idx])
      for ent in doc_bc5cdr.entities:
        dict_bc5cdr_entities[ent.type].append(ent.text)
      df['Named_Entities_genia_w_BC5CDR'][idx] = dict_bc5cdr_entities
    except (AssertionError, RuntimeError):
      continue
  
  return df

In [ ]:
# download and initialize a mimic pipeline with an i2b2 NER model
stanza.download('en', package='mimic', processors={'ner': 'i2b2'})
nlp_i2b2 = stanza.Pipeline('en', package='mimic', processors={'ner': 'i2b2'})

stanza.download('en', package='genia', processors={'ner': 'BC5CDR'})
nlp_bc5cdr = stanza.Pipeline('en', package='genia', processors={'ner': 'BC5CDR'})


2021-06-13 15:24:34 INFO: Downloading these customized packages for language: en (English)...
| Processor       | Package |
-----------------------------
| tokenize        | mimic   |
| pos             | mimic   |
| lemma           | mimic   |
| depparse        | mimic   |
| ner             | i2b2    |
| pretrain        | mimic   |
| backward_charlm | mimic   |
| forward_charlm  | mimic   |



2021-06-13 15:25:31 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-06-13 15:25:31 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | mimic   |
| pos       | mimic   |
| lemma     | mimic   |
| depparse  | mimic   |
| ner       | i2b2    |

2021-06-13 15:25:31 INFO: Use device: gpu
2021-06-13 15:25:31 INFO: Loading: tokenize


2021-06-13 15:25:37 INFO: Loading: pos
2021-06-13 15:25:37 INFO: Loading: lemma
2021-06-13 15:25:38 INFO: Loading: depparse
2021-06-13 15:25:38 INFO: Loading: ner
2021-06-13 15:25:38 INFO: Done loading processors!


2021-06-13 15:25:38 INFO: Downloading these customized packages for language: en (English)...
| Processor       | Package |
-----------------------------
| tokenize        | genia   |
| pos             | genia   |
| lemma           | genia   |
| depparse        | genia   |
| ner             | bc5cdr  |
| pretrain        | genia   |
| backward_charlm | pubmed  |
| forward_charlm  | pubmed  |



2021-06-13 15:26:36 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-06-13 15:26:36 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | genia   |
| pos       | genia   |
| lemma     | genia   |
| depparse  | genia   |
| ner       | bc5cdr  |

2021-06-13 15:26:36 INFO: Use device: gpu
2021-06-13 15:26:36 INFO: Loading: tokenize
2021-06-13 15:26:36 INFO: Loading: pos


2021-06-13 15:26:36 INFO: Loading: lemma
2021-06-13 15:26:36 INFO: Loading: depparse
2021-06-13 15:26:36 INFO: Loading: ner
2021-06-13 15:26:37 INFO: Done loading processors!


In [ ]:
medline_abstracts_lst = ["neoplasms_Medline.xlsx", \
                         "developmental_anomaly_Medline.xlsx", \
                         "certain_conditions_originating_in_the_perinatal_period_Medline.xlsx", \
                         "certain_infectious_or_parasitic_diseases_Medline", \
                         "diseases_of_the_blood_and_blood_forming_organs_Medline.xlsx", \
                         "disorders_involving_the_immune_mechanism_Medline", \
                         "injury_poisoning_or_certain_other_consequences_of_external_cause_Medline.xlsx", \
                         "pregnancy_childbirth_and_the_puerperium_Medline.xlsx"]

In [ ]:
icd_chapter_fname = "injury_poisoning_or_certain_other_consequences_of_external_cause_Medline.xlsx"

df_final = _extract_named_entities(icd_chapter_fname, nlp_i2b2, nlp_bc5cdr)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df_final.head()

,PMID,Title,Abstract,Named_Entities_mimic_w_i2b2,Named_Entities_genia_w_BC5CDR
0,33740791,Superficial Cervical Plexus Block for Awake La...,Pregnant patients with high-risk conditions in...,"{'PROBLEM': ['high-risk conditions', 'abnormal...","{'DISEASE': ['abnormal placentation', 'cardiov..."
1,33740772,Acquiring photoacoustic signature of hematocri...,Optical penetration inside human skin is const...,{'PROBLEM': ['Optical penetration inside human...,{}
2,33740763,Patterns of median nerve branching in the cubi...,OBJECTIVE: The purpose of this study was to de...,"{'TEST': ['this study', 'the PPT', 'PPT', 'DPT...","{'DISEASE': ['AIN', 'AIN', 'AIN', 'AIN', 'AIN'..."
3,33740699,"The ""Shamrock method"" for ultrasound-guided lu...",NaN,,
4,33740560,Gastrointestinal dysfunction in neuroinflammat...,Disorders of the nervous system can produce a ...,{'PROBLEM': ['Disorders of the nervous system'...,{'DISEASE': ['Disorders of the nervous system'...


In [ ]:
DATA_PATH = "medline_abstracts_w_entities"

if not os.path.exists(DATA_PATH):
  os.mkdir(DATA_PATH)

output_fname = icd_chapter_fname.replace('.xlsx', '')
pk.dump(df_final, open(os.path.join(DATA_PATH, f"{output_fname}.pk"), "wb"))

## Read back the pickle file to verify

In [ ]:
DATA_PATH = "medline_abstracts_w_entities"
icd_chapter_fname = "neoplasms_Medline"

output_fname = icd_chapter_fname.replace('.xlsx', '')
df = pk.load( open(os.path.join(DATA_PATH, f"{output_fname}.pk"), "rb"))

In [ ]:
df.head()

,PMID,Title,Abstract,Named_Entities_mimic_w_i2b2,Named_Entities_genia_w_BC5CDR
0,33706449,[Short-term and long-term outcomes of tricuspi...,Objective: To examine the short-term and long-...,"{'TREATMENT': ['tricuspid valve replacement', ...","{'DISEASE': ['left ventricular dysfunction', '..."
1,33706443,[A prognostic model of intrahepatic cholangioc...,Objective: To examine a survival prognostic mo...,"{'TREATMENT': ['a survival prognostic model', ...",{'DISEASE': ['intrahepatic cholangiocarcinoma'...
2,33706442,[Conversion therapy of biliary tract cancer fr...,Biliary tract cancer is found in the middle an...,"{'PROBLEM': ['Biliary tract cancer', 'Biliary ...","{'DISEASE': ['Biliary tract cancer', 'Biliary ..."
3,33706440,[Attach importance to the standardized diagnos...,"Gallbladder carcinoma,characterized by conceal...","{'PROBLEM': ['Gallbladder carcinoma', 'gallbla...","{'DISEASE': ['Gallbladder carcinoma', 'gallbla..."
4,33706424,Quality of life fluctuations before and after ...,NaN,,


In [ ]:
df['Named_Entities_genia_w_BC5CDR'].values[:20]

array([defaultdict(<class 'list'>, {'DISEASE': ['left ventricular dysfunction', 'left ventricular dysfunction', 'liver disease', 'liver disease', 'organ failure', 'low cardiac output syndrome', 'LCOS', 'LCOS', 'deaths', 'cardiogenic', 'death', 'anticoagulant complications', 'death', 'lung cancer', 'left ventricular dysfunction']}),
       defaultdict(<class 'list'>, {'DISEASE': ['intrahepatic cholangiocarcinoma', 'intrahepatic bile duct stones', 'chronic viral hepatitis', 'jaundice', 'intrahepatic bile duct stones', 'liver capsule invasion', 'intrahepatic bile duct stones']}),
       defaultdict(<class 'list'>, {'DISEASE': ['Biliary tract cancer', 'Biliary tract cancer']}),
       defaultdict(<class 'list'>, {'DISEASE': ['Gallbladder carcinoma', 'gallbladder cancer', 'cancer', 'gallbladder cancer']}),
       '',
       defaultdict(<class 'list'>, {'DISEASE': ['lung cancer', 'lung cancer']}),
       defaultdict(<class 'list'>, {'DISEASE': ['Lung cancer', 'cancer', 'death', 'lung cancer'

In [ ]:
df['Named_Entities_mimic_w_i2b2'].values[:20]

array([defaultdict(<class 'list'>, {'TREATMENT': ['tricuspid valve replacement', 'TVR', 'TVR', 'Cardiovascular Surgery', 'The long-term survival rate', 'continuous renal replacement therapy', 'LCOS', 'TVR'], 'PROBLEM': ['left ventricular dysfunction', 'left ventricular dysfunction', 'end-staged liver disease score', 'end-staged liver disease score', 'multiple organ failure', 'low cardiac output syndrome', 'cardiogenic cause', 'anticoagulant complications', '1 death', 'lung cancer', 'left ventricular dysfunction', 'morbidity'], 'TEST': ['The clinical data', 'The preoperative ejection fraction', 'Charlson comorbidity index', 'cardiac operative risk evaluation', 'EuroSCORE', 'The EuroSCORE', 'survival rates']}),
       defaultdict(<class 'list'>, {'TREATMENT': ['a survival prognostic model', 'curative intent resection', 'Cox proportional hazard model', 'curative intent resection'], 'PROBLEM': ['intrahepatic cholangiocarcinoma', 'intrahepatic bile duct stones', 'chronic viral hepatitis', '

### The following cell is for extracting named entities from a random clinical note. It's not related to this jupyter notebook's project

In [ ]:
i2b2_model = nlp_i2b2
clinical_text = "[Redacted] is an 86-year-old male with a history of coronary artery disease status post CABG, heart failure, AFib on Eliquis, dementia presenting with weakness.  History obtained from the patient's wife is the patient is altered and not able to provide a complete history.  She states that he noted bilateral extremity weakness and was not able to walk today.  This seems to been slowly progressing over time.  She also notes he is becoming progressively more altered and nothing he sees the dog and is talking to people who are not there.  She notes similar symptoms when he has had urinary tract infections.  She denies any trauma.  She notes compliance with medications.  No known history of strokes.  She denies any facial asymmetry, aphasia or unilateral weakness.  He has had multiple admissions to Cook County in Northwestern for “nasty “urinary tract infections.  He is not currently taking antibiotics."
dict_i2b2_entities = defaultdict(list)

doc_i2b2 = i2b2_model(clinical_text)
for ent in doc_i2b2.entities:
  dict_i2b2_entities[ent.type].append(ent.text)

In [ ]:
dict_i2b2_entities

defaultdict(list,
            {'PROBLEM': ['coronary artery disease',
              'heart failure',
              'AFib',
              'dementia',
              'weakness',
              'bilateral extremity weakness',
              'similar symptoms',
              'urinary tract infections',
              'any trauma',
              'strokes',
              'any facial asymmetry',
              'aphasia',
              'unilateral weakness',
              'urinary tract infections'],
             'TREATMENT': ['CABG', 'Eliquis', 'medications', 'antibiotics']})